In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("sqlite_db_pythonsqlite.db")
cur = conn.cursor()

In [3]:
#using pandas to view the query results

In [4]:
q = "Select * from Bookings"
bookings_df = pd.read_sql_query(q, conn)
bookings_df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [5]:
q = "Select * from Facilities"
Facilities_df = pd.read_sql_query(q, conn)
Facilities_df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [6]:
q = "Select * from Members"
Members_df = pd.read_sql_query(q, conn)
Members_df.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


#### /* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [7]:
q10 = "SELECT  name, SUM(totalcost) as revenue \
FROM (SELECT f.membercost, f.guestcost, b.memid, b.slots, b.facid, f.name, \
        CASE b.memid \
            WHEN 0 THEN f.guestcost * b.slots \
            ELSE f.membercost * b.slots \
            END totalcost \
    FROM Bookings AS b \
    LEFT JOIN Facilities AS f ON b.facid = f.facid) as tcost \
GROUP BY facid \
HAVING SUM(totalcost) < 1000 \
ORDER BY revenue"

In [8]:
pd.read_sql_query(q10, conn)

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


#### /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


In [9]:
q11 = "Select m1.surname || ', ' || m1.firstname as recomendee, m2.recommender \
        FROM Members as m1 \
        INNER JOIN (SELECT memid, surname || ', ' || firstname as recommender \
                    FROM Members) as m2 \
        ON m1.recommendedby = m2.memid \
        ORDER BY recomendee"

In [10]:
pd.read_sql_query(q11, conn)

,recomendee,recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


#### /* Q12: Find the facilities with their usage by member, but not guests */

In [11]:
#I'm not sure exactly what "usage by member" means here, so for the sake of the assignment, I've assumed it means
#how many members (not guests) have used that facility. 

In [12]:
q12 = "SELECT name, COUNT(memid) as memberusage \
        FROM Bookings \
        LEFT JOIN Facilities \
            ON Bookings.facid = Facilities.facid \
        WHERE memid <> 0 \
        GROUP BY name \
        ORDER BY name"

In [13]:
pd.read_sql_query(q12, conn)

,name,memberusage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


#### /* Q13: Find the facilities usage by month, but not guests */

In [14]:
#I am solving this question  based on the assumption that I am doing what I did above in q12, 
#but split by month instead of facility

In [15]:
q13 = "SELECT strftime('%m', starttime) as month, COUNT(memid) as memberusage \
        FROM Bookings \
        LEFT JOIN Facilities \
            ON Bookings.facid = Facilities.facid \
        WHERE memid <> 0 \
        GROUP BY strftime('%m', starttime)"

In [16]:
pd.read_sql_query(q13, conn, parse_dates=True)

,month,memberusage
0,07,480
1,08,1168
2,09,1512
